In [9]:
from bs4 import BeautifulSoup

ex1 = '''
<html>
    <head>
        <title> HTML 연습 </title>
    </head>
    <body>
        <p align="center"> text1 </p>
        <p align="right"> text2 </p>
        <p align="left"> text3 </p>
        <img src="C:\\Users\\Hyewon\\Pictures\\배경화면\\220906_wide.jpg">
    </body>
</html>
    '''

In [10]:
soup = BeautifulSoup(ex1, 'html.parser')
# soup.find('태그명', 속성명 = '속성값')
soup.find('p', align='right').get_text().strip()

'text2'

In [11]:
soup.find_all('p')

[<p align="center"> text1 </p>,
 <p align="right"> text2 </p>,
 <p align="left"> text3 </p>]

In [12]:
for i in soup.find_all('p'):
    print(i.get_text().strip().replace('text', 'txt'))

txt1
txt2
txt3


In [13]:
x = soup.find_all('p')
for idx, i in enumerate(x, start=1):
    print(idx, i.get_text().strip())

1 text1
2 text2
3 text3


In [ ]:
# 웹 제어를 위한 크롬드라이버 설정
# 크롬 드라이버 자동 다운로드
from selenium import webdriver
import chromedriver_autoinstaller as ca
driver = webdriver.Chrome(ca.install())

# 웹 제어를 위한 크롬드라이버 설정 2 
# 크롬드라이버 수동 다운로드 및 활용
from selenium import webdriver
driver = webdriver.Chrome('c:/Users/Hyewon/BigLeader/Crawler/chromedriver.exe')

# 웹 제어를 위한 크롬드라이버 설정 3
# 파이썬 3.10 이상 버전
from selenium import webdriver
driver = webdriver.Chrome()

In [2]:
# 논문 제목 긁어오기

from selenium import webdriver
from selenium.webdriver.common.by import By
import time, os, sys

driver = webdriver.Chrome()  # chrome 제어  # python 3.10 이하에서는 Chrome()안에 인자(chromedriver_autoinstaller.install()) 넣어줘야 함

# 웹사이트 접속하기
driver.get('https://www.riss.kr')
time.sleep(2)

# 팝업창 닫기
main = driver.window_handles
for handle in main:
    if handle != main[0]:
        driver.switch_to.window(handle)
        driver.close()

# 원래 창으로 돌아가기
driver.switch_to.window(main[0])
driver.maximize_window()  # 화면 최대로 만들기

# 검색창에 검색어 넣기

# 검색어 넣기
time.sleep(2)
driver.find_element(By.ID, 'query').send_keys('빅데이터' + '\n')

# 학위 논문 클릭하기
time.sleep(2)
driver.find_element(By.LINK_TEXT, '학위논문').click()
time.sleep(2)

# 논문의 제목 크롤링
from bs4 import BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser') 

# 웹사이트 닫기
time.sleep(5)
driver.close()

In [5]:
title = soup.find('div', class_='srchResultListW').find_all('p', class_='title')
for idx, i in enumerate(title):
    print(idx, i.get_text())

0 빅데이터 산업의 활성화에 따른 개인정보 보호에 관한 연구
1 빅데이터 활용을 위한 법제 개선 방안에 관한 연구
2 빅데이터 분석을 활용한 지역맞춤형 평생교육프로그램 개발 모형
3 기업의 빅데이터 활용의도에 영향을 미치는 요인에 관한 연구
4 유동인구 빅데이터의 효율적 활용 방안에 관한 연구 : C통신사 사례를 중심으로
5 빅데이터 환경에서 개인정보 이용에 관한 법적 고찰
6 빅데이터로서 수사기록 보존폐기에 대한 연구
7 특허분석을 통한 빅데이터의 공백기술 예측 및 기술개발 방향에 대한 연구
8 빅 데이터를 활용한 패션 브랜드 웹기반 소비자 평가 트렌드 추이 분석 : 브랜드 마케팅 전략과의 연계성을 중심으로
9 빅데이터 활용을 위한 개인정보 비식별화 관련 지식재산 및 정보보호법률 개선방안


In [6]:
# 게시물 제목만 추출
논문 = soup.find('div', class_='srchResultListW')

제목 = 논문.find_all('p', class_='title')
저자 = 논문.find_all('span', class_='writer')
소속 = 논문.find_all('span', class_='assigned')

for i in 소속:
    print(i.get_text())

전북대학교
동국대학교
대구한의대학교 대학원
숭실대학교 대학원
목포대학교 경영행정대학원
고려대학교 대학원
서울대학교 융합과학기술대학원
고려대학교 공학대학원
국민대학교 일반대학원
고려대학교 공학대학원


In [7]:
# 저장하기 기초 - 표준 출력 방향을 전환해서 저장하기
import sys
f_name = 'c:/Users/Hyewon/BigLeader/Crawler/day2/riss.txt'

원본 = sys.stdout  # 원래 기능 잠시 저장

with open(f_name, 'a', encoding='UTF-8') as file :   # a => append : 이어쓰기 , w => write  
    sys.stdout = file

    for i in 제목:
        print(i.get_text().strip())

sys.stdout = 원본 

print('데이터 수집이 정상적으로 완료되었습니다.')
print('수집된 결과는 %s 에 저장되었습니다' %f_name)


      


데이터 수집이 정상적으로 완료되었습니다.
수집된 결과는 c:/Users/Hyewon/BigLeader/Crawler/day2/riss.txt 에 저장되었습니다


네이버 이미지 크롤러 제작

In [35]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import urllib.request
import time, os

# 사용자에게 필요한 정보 입력 받기
query_txt = input('1. 크롤링할 이미지의 키워드는 : ')
cnt = int(input('2. 크롤링 할 건수는? '))

# 현재 시간 활용 폴더 생성
now = 'C:/Users/Hyewon/BigLeader/Crawler/day2/' + time.strftime("%Y%m%d_%H%M%S") + "-" + query_txt
os.makedirs(now)
os.chdir(now)

# 크롬 드라이버를 사용해 웹 브라우저 실행
driver = webdriver.Chrome()

base_link = 'https://search.naver.com/search.naver?where=image&sm=tab_jum&query='
driver.get(base_link + query_txt)

# 웹 페이지를 6회 스크롤 다운(End 키 전송) - 네이버 이미지는 총 6회 스크롤 다운이 최대
for i in range(6):
    time.sleep(1)
    driver.find_element(By.XPATH, '//body').send_keys(Keys.END)

# 이미지 추출
soup = BeautifulSoup(driver.page_source, 'html.parser')
img_src = []

for i in soup.find_all('img', class_='_image _listImage'):
    img_src.append(i["src"])

# print(img_src)

for idx, save_img in enumerate(img_src, start=1):
    urllib.request.urlretrieve(save_img, str(idx) + ".jpg")
    if idx == cnt:
        break
    
driver.close()


<img alt="광주강아지파양은 안락사 없고 쾌적한 보호소" class="_image _listImage" data-org-height="321" data-org-width="340" src="https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMzA2MDhfNDEg%2FMDAxNjg2MTkxMzI2MDE2.iYRuN2r7AT_1Drn3cEo4uuvFvCtO6eEbzBQxR8SMI5kg.1_PhlIlqx_iClyrNtOgQAGDHMANhYD832aUY1KcgrJ0g.JPEG.genisu%2F%25B0%25AD%25BE%25C6%25C1%25F6806.jpg&amp;type=a340" style="height: 206px; opacity: 1;"/>
<img alt="강아지 분양 받을 때 꼭 알아야 할 것" class="_image _listImage" data-org-height="337" data-org-width="340" src="https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMzA2MDhfMjYw%2FMDAxNjg2MjMwODQ2MzIx.8mQvsH7I1WmFVoYL2zTibVEUuilnFv3Bmt-QkQG6uAwg.gHIB1XAXegi_m3eCfQP4kwl8s4XdJZjba3S7-zmGc7Eg.JPEG.caries79%2F1686230845390.jpg&amp;type=a340" style="height: 216px; opacity: 1;"/>
<img alt="강아지 사람 나이 계산법!" class="_image _listImage" data-org-height="241" data-org-width="340" src="https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMzA2MDdf

In [6]:
# 함수화
def user_img(name, no) :
    # #Step 2. 사용자에게 필요한 정보를 입력 받습니다.
    # 현재시간 활용 폴더 생성
    now = "c:/py_temp/" + time.strftime("%Y%m%d_%H%M%S") +'-' + name
    os.makedirs(now)
    os.chdir(now)
    # Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
    try :
        driver = webdriver.Chrome()
        driver.get(f'https://search.naver.com/search.naver?where=image&sm=tab_jum&query={name}')
    except :
        pass
    # 웹페이지를 6회 스크롤 다운 합니다 (END 키 전송)
    for _ in range(6) :
        time.sleep(3)
        driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
    # Step 4. 이미지 추출하기
    img_src = []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    for i in soup.find_all('img', '_image _listImage') :
        img_src.append(i['src'])
    for idx, save_img in enumerate(img_src, start = 1) :
        urllib.request.urlretrieve(save_img, str(idx) + ".png")
        if idx == no :
            break
    driver.close()